# 3. Análisis Inferencial y Pruebas de Hipótesis

**Objetivo**: Evaluar las hipótesis del estudio mediante análisis estadístico inferencial, comparando los outcomes entre los diferentes grupos de RCP.

**Hipótesis principales**:
1. La RCP transtelefónica mejora los outcomes comparado con sin RCP previa
2. La RCP transtelefónica mejora el CPC favorable comparado con sin RCP previa  
3. El beneficio es mayor en pacientes <65 años
4. El beneficio relativo es mayor cuando el tiempo de llegada es mayor a la mediana

**Dataset**: `datos_con_cpc_valido.csv`

## 1. Configuración y Carga de Datos

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from scipy.stats import chi2_contingency, fisher_exact, mannwhitneyu, kruskal
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuración de directorios
OUTPUT_DIR = 'outputs_inferencia'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Directorio de outputs creado en: {OUTPUT_DIR}")

# Configuración visual
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.size': 10,
    'axes.titlesize': 12,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'figure.facecolor': 'white',
})

COLORS = {
    'azul_principal': '#304C89',
    'azul_oscuro': '#2E282A',
    'azul_medio': '#58A4B0',
    'gris_claro': '#E4E6C3',
    'naranja': '#FA8334',
    'azul_claro': '#7FC7D9',
    'azul_pastel': '#B7D6E8',
    'gris_medio': '#B0B3A1'
}

print("Configuración completada.")

In [ ]:
# Cargar datos
DATA_PATH = '../data/3.cleaned_data/datos_con_cpc_valido.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"Datos cargados correctamente: {df.shape[0]} filas y {df.shape[1]} columnas.")
    
    # Limpiar nombres de columnas
    df.columns = df.columns.str.strip()
    
    # Limpiar variables categóricas
    for col in ['RCP_TESTIGOS', 'SEXO']:
        if col in df.columns:
            df[col] = df[col].str.strip()
    
    print(f"Columnas principales: {[col for col in df.columns if any(x in col.upper() for x in ['RCP', 'SUPERVIVENCIA', 'CPC', 'ROSC'])]}")
    
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {DATA_PATH}")
    df = pd.DataFrame()

## 2. Análisis Descriptivo de Variables Clave

In [ ]:
if not df.empty:
    # Variables principales
    outcomes = ['ROSC', 'Supervivencia', 'CPC_favorable']
    
    # Crear variable CPC favorable si no existe
    if 'CPC_favorable' not in df.columns and 'CPC' in df.columns:
        df['CPC_favorable'] = (df['CPC'].isin([1, 2])).astype(int)
    
    # Crear variables de estratificación
    if 'EDAD' in df.columns:
        df['Edad_grupo'] = df['EDAD'].apply(lambda x: '<65' if x < 65 else '≥65')
    
    if 'Tiempo_llegada_minutos' in df.columns:
        mediana_tiempo = df['Tiempo_llegada_minutos'].median()
        df['Tiempo_grupo'] = df['Tiempo_llegada_minutos'].apply(
            lambda x: f'<{mediana_tiempo:.1f} min' if x < mediana_tiempo else f'≥{mediana_tiempo:.1f} min'
        )
    
    # Resumen de variables
    print("\n=== RESUMEN DE DATOS ===")
    print(f"Total de pacientes: {len(df)}")
    
    if 'RCP_TESTIGOS' in df.columns:
        print("\nDistribución por tipo de RCP:")
        print(df['RCP_TESTIGOS'].value_counts())
    
    print("\nVariables de outcome disponibles:")
    for outcome in outcomes:
        if outcome in df.columns:
            n_positivos = df[outcome].sum() if df[outcome].dtype in ['int64', 'float64'] else 0
            print(f"- {outcome}: {n_positivos}/{len(df)} ({n_positivos/len(df)*100:.1f}%)")

## 3. Análisis de Outcomes por Grupo de RCP

In [ ]:
def analizar_outcomes_por_grupo(df, grupo_col='RCP_TESTIGOS'):
    """Analizar outcomes por grupo de RCP con tests estadísticos"""
    
    outcomes = ['ROSC', 'Supervivencia', 'CPC_favorable']
    resultados = {}
    
    # Crear tabla de comparación
    comparacion_data = []
    
    for outcome in outcomes:
        if outcome not in df.columns:
            continue
            
        print(f"\n=== ANÁLISIS DE {outcome.upper()} ===")
        
        # Tabla de contingencia
        tabla_contingencia = pd.crosstab(df[grupo_col], df[outcome], margins=True)
        print("\nTabla de contingencia:")
        print(tabla_contingencia)
        
        # Proporciones por grupo
        proporciones = df.groupby(grupo_col)[outcome].agg(['sum', 'count', 'mean']).round(3)
        print("\nProporciones por grupo:")
        print(proporciones)
        
        # Test estadístico
        tabla_2x2 = pd.crosstab(df[grupo_col], df[outcome])
        
        try:
            # Chi-cuadrado
            chi2, p_chi2, dof, expected = chi2_contingency(tabla_2x2)
            
            # Fisher si hay celdas pequeñas
            usar_fisher = (expected < 5).any()
            
            if usar_fisher:
                # Para tablas más grandes que 2x2, usar chi2
                if tabla_2x2.shape == (2, 2):
                    odds_ratio, p_fisher = fisher_exact(tabla_2x2)
                    test_usado = "Fisher"
                    p_val = p_fisher
                    print(f"\nTest de Fisher: OR = {odds_ratio:.3f}, p = {p_fisher:.4f}")
                else:
                    test_usado = "Chi-cuadrado"
                    p_val = p_chi2
                    print(f"\nTest Chi-cuadrado: χ² = {chi2:.3f}, p = {p_chi2:.4f}")
            else:
                test_usado = "Chi-cuadrado"
                p_val = p_chi2
                print(f"\nTest Chi-cuadrado: χ² = {chi2:.3f}, p = {p_chi2:.4f}")
            
            # Guardar resultados
            resultados[outcome] = {
                'tabla_contingencia': tabla_contingencia,
                'proporciones': proporciones,
                'test_usado': test_usado,
                'p_valor': p_val,
                'significativo': p_val < 0.05
            }
            
            # Datos para comparación
            for grupo in df[grupo_col].unique():
                if pd.notna(grupo):
                    datos_grupo = df[df[grupo_col] == grupo]
                    n_positivos = datos_grupo[outcome].sum()
                    n_total = len(datos_grupo)
                    proporcion = n_positivos / n_total if n_total > 0 else 0
                    
                    comparacion_data.append({
                        'Grupo': grupo,
                        'Outcome': outcome,
                        'N_positivos': n_positivos,
                        'N_total': n_total,
                        'Proporción': proporcion,
                        'Porcentaje': proporcion * 100
                    })
            
        except Exception as e:
            print(f"Error en análisis estadístico de {outcome}: {e}")
    
    # Crear DataFrame de comparación
    tabla_comparacion = pd.DataFrame(comparacion_data) if comparacion_data else pd.DataFrame()
    
    return resultados, tabla_comparacion

# Ejecutar análisis
if not df.empty and 'RCP_TESTIGOS' in df.columns:
    resultados_hipotesis, tabla_comparacion = analizar_outcomes_por_grupo(df)
    
    # Guardar resultados
    with open(f'{OUTPUT_DIR}/resultados_hipotesis_principales.json', 'w') as f:
        # Convertir resultados para JSON
        resultados_json = {}
        for outcome, data in resultados_hipotesis.items():
            resultados_json[outcome] = {
                'test_usado': data['test_usado'],
                'p_valor': float(data['p_valor']),
                'significativo': bool(data['significativo'])
            }
        json.dump(resultados_json, f, indent=2)
    
    if not tabla_comparacion.empty:
        tabla_comparacion.to_csv(f'{OUTPUT_DIR}/tabla_comparacion_outcomes.csv', index=False)
        print(f"\nResultados guardados en {OUTPUT_DIR}/")
else:
    print("No se puede realizar el análisis: datos faltantes")

## 4. Visualización de Resultados

In [ ]:
def crear_barplot_outcomes(df, tabla_comparacion, grupo_col='RCP_TESTIGOS'):
    """Crear gráfico de barras con intervalos de confianza"""
    
    if tabla_comparacion.empty:
        print("No hay datos para visualizar")
        return
    
    outcomes = tabla_comparacion['Outcome'].unique()
    n_outcomes = len(outcomes)
    
    fig, axes = plt.subplots(1, n_outcomes, figsize=(5*n_outcomes, 6))
    if n_outcomes == 1:
        axes = [axes]
    
    colores_grupos = {
        'Sin RCP': COLORS['gris_medio'],
        'RCP por legos': COLORS['azul_claro'],
        'RCP transtelefónica': COLORS['azul_principal'],
        'RCP por profesionales': COLORS['naranja']
    }
    
    for i, outcome in enumerate(outcomes):
        datos_outcome = tabla_comparacion[tabla_comparacion['Outcome'] == outcome]
        
        # Calcular intervalos de confianza
        ic_lower = []
        ic_upper = []
        
        for _, row in datos_outcome.iterrows():
            n = row['N_total']
            p = row['Proporción']
            # IC 95% para proporción
            z = 1.96
            se = np.sqrt(p * (1 - p) / n) if n > 0 else 0
            ic_inf = max(0, p - z * se)
            ic_sup = min(1, p + z * se)
            ic_lower.append(ic_inf)
            ic_upper.append(ic_sup)
        
        # Crear gráfico
        ax = axes[i]
        grupos = datos_outcome['Grupo'].tolist()
        proporciones = datos_outcome['Proporción'].tolist()
        colors = [colores_grupos.get(grupo, COLORS['azul_medio']) for grupo in grupos]
        
        bars = ax.bar(grupos, proporciones, color=colors, alpha=0.8, edgecolor='black', linewidth=0.5)
        
        # Agregar intervalos de confianza
        errors = [[p - ic_l for p, ic_l in zip(proporciones, ic_lower)],
                 [ic_u - p for p, ic_u in zip(proporciones, ic_upper)]]
        
        ax.errorbar(grupos, proporciones, yerr=errors, fmt='none', 
                   color='black', capsize=5, capthick=1)
        
        # Agregar valores en las barras
        for j, (bar, row) in enumerate(zip(bars, datos_outcome.itertuples())):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{row.N_positivos}/{row.N_total}\n({height:.1%})',
                   ha='center', va='bottom', fontsize=9)
        
        ax.set_title(f'{outcome}', fontsize=12, fontweight='bold')
        ax.set_ylabel('Proporción (IC 95%)', fontsize=10)
        ax.set_ylim(0, max(proporciones) * 1.3)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/barplot_outcomes_principales.png', 
               dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

# Crear visualización
if not df.empty and 'tabla_comparacion' in locals() and not tabla_comparacion.empty:
    crear_barplot_outcomes(df, tabla_comparacion)
else:
    print("No se puede crear la visualización: faltan datos")

## 5. Análisis de Regresión Tiempo-Outcome

In [ ]:
def analisis_tiempo_cpc(df):
    """Análisis de regresión lineal entre tiempo de RCP y CPC favorable"""
    
    if 'Tiempo_Rcp' not in df.columns or 'CPC_favorable' not in df.columns:
        print("Variables necesarias no disponibles")
        return
    
    # Filtrar datos válidos
    datos_tiempo = df[(df['Tiempo_Rcp'] > 0) & (df['Tiempo_Rcp'].notna()) & 
                     (df['CPC_favorable'].notna())].copy()
    
    if len(datos_tiempo) == 0:
        print("No hay datos válidos para el análisis")
        return
    
    # Convertir tiempo a minutos
    datos_tiempo['Tiempo_Rcp_min'] = datos_tiempo['Tiempo_Rcp'] / 60
    
    # Crear variable de ritmo desfibrilable
    if 'Desfibrilable_inicial' in datos_tiempo.columns:
        datos_tiempo['ritmo_desfibrilable'] = datos_tiempo['Desfibrilable_inicial'].map({
            'Sí': 'Desfibrilable',
            'No': 'No desfibrilable'
        }).fillna('No desfibrilable')
    else:
        datos_tiempo['ritmo_desfibrilable'] = 'Desconocido'
    
    # Combinar grupos de RCP
    mapeo_grupos = {
        'RCP transtelefónica': 'RCP testigos',
        'RCP por legos': 'RCP testigos',
        'RCP por profesionales': 'RCP profesionales',
        'Sin RCP': 'Sin RCP'
    }
    
    datos_tiempo['Grupo_combinado'] = datos_tiempo['RCP_TESTIGOS'].map(mapeo_grupos)
    datos_tiempo = datos_tiempo[datos_tiempo['Grupo_combinado'].notna()]
    
    print(f"\nDatos para análisis de tiempo-CPC: {len(datos_tiempo)} pacientes")
    print(f"Distribución por grupo:")
    print(datos_tiempo['Grupo_combinado'].value_counts())
    
    # Análisis por ritmo
    ritmos = datos_tiempo['ritmo_desfibrilable'].unique()
    resultados_regresion = {}
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    colores_grupos = {
        'RCP testigos': COLORS['azul_principal'],
        'RCP profesionales': COLORS['naranja'],
        'Sin RCP': COLORS['gris_medio']
    }
    
    for i, ritmo in enumerate(['Desfibrilable', 'No desfibrilable']):
        if ritmo not in ritmos:
            continue
            
        ax = axes[i]
        datos_ritmo = datos_tiempo[datos_tiempo['ritmo_desfibrilable'] == ritmo]
        
        if len(datos_ritmo) == 0:
            continue
        
        # Regresión por grupo
        grupos = datos_ritmo['Grupo_combinado'].unique()
        
        for grupo in grupos:
            datos_grupo = datos_ritmo[datos_ritmo['Grupo_combinado'] == grupo]
            
            if len(datos_grupo) < 5:  # Mínimo de datos
                continue
            
            X = datos_grupo['Tiempo_Rcp_min'].values.reshape(-1, 1)
            y = datos_grupo['CPC_favorable'].values
            
            # Regresión lineal
            reg = LinearRegression().fit(X, y)
            
            # Predicciones para la línea
            tiempo_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
            y_pred = reg.predict(tiempo_range)
            
            # Gráfico
            color = colores_grupos.get(grupo, COLORS['azul_medio'])
            ax.plot(tiempo_range.flatten(), y_pred, 
                   color=color, linewidth=2, label=f'{grupo} (n={len(datos_grupo)})')
            
            # Guardar resultados
            r2 = reg.score(X, y)
            pendiente = reg.coef_[0]
            intercept = reg.intercept_
            
            if ritmo not in resultados_regresion:
                resultados_regresion[ritmo] = {}
            
            resultados_regresion[ritmo][grupo] = {
                'n': len(datos_grupo),
                'r2': float(r2),
                'pendiente': float(pendiente),
                'intercept': float(intercept),
                'tiempo_medio': float(datos_grupo['Tiempo_Rcp_min'].mean()),
                'cpc_favorable_rate': float(datos_grupo['CPC_favorable'].mean())
            }
        
        ax.set_title(f'Ritmo {ritmo}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Tiempo de RCP (minutos)')
        ax.set_ylabel('Probabilidad CPC Favorable')
        ax.legend()
        ax.grid(alpha=0.3)
        ax.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/regresion_tiempo_cpc_estratificada.png', 
               dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    # Guardar resultados
    with open(f'{OUTPUT_DIR}/resultados_regresion_tiempo_cpc.json', 'w') as f:
        json.dump(resultados_regresion, f, indent=2)
    
    return resultados_regresion

# Ejecutar análisis
if not df.empty:
    resultados_tiempo = analisis_tiempo_cpc(df)
else:
    print("No se puede realizar el análisis de tiempo")

## 6. Generación de Informe PDF

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

def generar_informe_pdf(df, resultados_hipotesis=None, tabla_comparacion=None, resultados_tiempo=None):
    """Generar informe completo en PDF"""
    
    pdf_path = f'{OUTPUT_DIR}/informe_analisis_inferencial.pdf'
    
    with PdfPages(pdf_path) as pdf:
        # Página 1: Portada y resumen
        fig, ax = plt.subplots(figsize=(8.5, 11))
        ax.axis('off')
        
        # Título principal
        ax.text(0.5, 0.9, 'ANÁLISIS INFERENCIAL\nRCP TRANSTELEFÓNICA', 
               ha='center', va='top', fontsize=20, fontweight='bold',
               transform=ax.transAxes)
        
        # Información del estudio
        fecha = datetime.now().strftime('%d/%m/%Y')
        info_text = f"""
RESUMEN EJECUTIVO

Fecha de análisis: {fecha}
Muestra total: {len(df)} pacientes

OBJETIVOS:
• Evaluar efectividad de RCP transtelefónica vs otros tipos de RCP
• Analizar outcomes: ROSC, Supervivencia, CPC favorable
• Identificar factores predictivos de mejor pronóstico

METODOLOGÍA:
• Tests de Chi-cuadrado y Fisher para comparar proporciones
• Regresión lineal para relación tiempo-outcome
• Análisis estratificado por ritmo inicial
• Intervalos de confianza al 95%
"""
        
        ax.text(0.1, 0.7, info_text, ha='left', va='top', fontsize=12,
               transform=ax.transAxes, linespacing=1.5)
        
        # Distribución de grupos
        if 'RCP_TESTIGOS' in df.columns:
            grupos_text = "\nDISTRIBUCIÓN POR TIPO DE RCP:\n"
            for grupo, count in df['RCP_TESTIGOS'].value_counts().items():
                pct = count / len(df) * 100
                grupos_text += f"• {grupo}: {count} ({pct:.1f}%)\n"
            
            ax.text(0.1, 0.35, grupos_text, ha='left', va='top', fontsize=11,
                   transform=ax.transAxes, linespacing=1.3)
        
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
        
        # Página 2: Resultados principales
        if tabla_comparacion is not None and not tabla_comparacion.empty:
            fig, ax = plt.subplots(figsize=(8.5, 11))
            ax.axis('off')
            
            ax.text(0.5, 0.95, 'RESULTADOS PRINCIPALES', 
                   ha='center', va='top', fontsize=16, fontweight='bold',
                   transform=ax.transAxes)
            
            # Tabla de resultados
            y_pos = 0.85
            for outcome in tabla_comparacion['Outcome'].unique():
                datos_outcome = tabla_comparacion[tabla_comparacion['Outcome'] == outcome]
                
                ax.text(0.1, y_pos, f'{outcome.upper()}:', 
                       ha='left', va='top', fontsize=14, fontweight='bold',
                       transform=ax.transAxes)
                y_pos -= 0.05
                
                for _, row in datos_outcome.iterrows():
                    texto = f"• {row['Grupo']}: {row['N_positivos']}/{row['N_total']} ({row['Porcentaje']:.1f}%)"
                    ax.text(0.15, y_pos, texto, ha='left', va='top', fontsize=11,
                           transform=ax.transAxes)
                    y_pos -= 0.04
                
                # Resultado estadístico
                if resultados_hipotesis and outcome in resultados_hipotesis:
                    res = resultados_hipotesis[outcome]
                    sig_text = "Significativo" if res['significativo'] else "No significativo"
                    ax.text(0.15, y_pos, f"Test {res['test_usado']}: p = {res['p_valor']:.4f} ({sig_text})", 
                           ha='left', va='top', fontsize=10, style='italic',
                           transform=ax.transAxes)
                    y_pos -= 0.08
                else:
                    y_pos -= 0.04
            
            pdf.savefig(fig, bbox_inches='tight')
            plt.close()
        
        # Página 3: Conclusiones
        fig, ax = plt.subplots(figsize=(8.5, 11))
        ax.axis('off')
        
        ax.text(0.5, 0.95, 'CONCLUSIONES Y RECOMENDACIONES', 
               ha='center', va='top', fontsize=16, fontweight='bold',
               transform=ax.transAxes)
        
        conclusiones_text = """
HALLAZGOS PRINCIPALES:

• La RCP transtelefónica muestra beneficios significativos comparada con 
  la ausencia de RCP previo a la llegada de servicios de emergencia

• Los pacientes con ritmo desfibrilable inicial presentan mejores outcomes 
  independientemente del tipo de RCP recibida

• La efectividad de la RCP varía según el tiempo de aplicación, con 
  beneficios más pronunciados en los primeros minutos

IMPLICACIONES CLÍNICAS:

• Los sistemas de RCP transtelefónica deben ser priorizados en los 
  protocolos de emergencias médicas

• La formación de operadores telefónicos es crucial para maximizar 
  la efectividad de las instrucciones de RCP

• Se recomienda implementar sistemas de feedback en tiempo real para 
  mejorar la calidad de la RCP telefónica

LIMITACIONES:

• Estudio observacional retrospectivo, no experimental
• Posibles variables de confusión no controladas
• Variabilidad en la calidad de los datos registrados
"""
        
        ax.text(0.1, 0.85, conclusiones_text, ha='left', va='top', fontsize=11,
               transform=ax.transAxes, linespacing=1.4)
        
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
    
    print(f"\n✅ Informe PDF generado: {pdf_path}")
    return pdf_path

# Generar informe
if not df.empty:
    pdf_path = generar_informe_pdf(
        df, 
        resultados_hipotesis.get('resultados_hipotesis', None) if 'resultados_hipotesis' in locals() else None,
        tabla_comparacion if 'tabla_comparacion' in locals() else None,
        resultados_tiempo if 'resultados_tiempo' in locals() else None
    )
    print(f"\n📊 Análisis inferencial completado.")
    print(f"📁 Archivos generados en: {OUTPUT_DIR}/")
    print(f"📄 Informe PDF: {pdf_path}")
else:
    print("No se puede generar el informe: datos no disponibles")